In [218]:
import numpy as np
import pandas as pd

Einlesen der Dateien

In [369]:
df_eval = pd.read_csv("./otv_data/Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-00_events.csv")
df_gt = pd.read_csv("./gt_data/Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-00.csv")

Funktionen:

In [404]:
#sortiert spalten nach Zeit
def sort_time(df, col = "DateTime"):
    df[col] = pd.to_datetime(df[col])
    filtered = df.sort_values(by=col)
    return filtered

#filtert df nach min und max zeitwert
def get_df_range(df, t_min, t_max, col = "DateTime" ):
        filtered = df[(df[col] >= t_min) & (df[col] < t_max)]
        return filtered

#überprüft ob 2 array die gleichen strings enthält
def array_similar(array1, array2):
      if not len(array1) == len(array2):
            raise ValueError("Arrays dont have the same dimesion!")
      for word in array1:
            if word in array2:
                  next
            else:
                  raise ValueError("Arrays dont contain the same strings")
                  break
      return True
            
#überprüft ob spalte identische Gates  haben
def get_gates(df1, df2, col = "SectionID"):
      df1 = df1[col].unique()
      df2 = df2[col].unique()
      if (array_similar(df1, df2)):
            return df1
      else:
            raise ValueError("Tabllen haben unterschiedliche Gates")

def get_value_counts(df, sections, section_index , value = "Class", section_name = "SectionID"):
      """holt value_counts aus df mit array sections nach index raus"""
      return df.loc[df[section_name] == sections[section_index], [value]].value_counts()

In [434]:
get_value_counts(df_12, gates, 1)

Class     
car           56
person        32
bicycle        2
bus            2
motorcycle     1
truck          1
dtype: int64

In [377]:
#gate import
gates = get_gates(df_eval, df_gt)

In [378]:
#equivalenz Tabelle
eq_tb = {
    "Person": "person",
    "Pkw ohne Anhänger": "car",
    "Pkw mit Anhänger": "car",
    "Motorisierte Zweiräder": "motorcycle",
    "Lieferwagen ohne Anhänger": "car",
    "Lieferwagen mit Anhänger": "car",
    "Lkw ohne Anhänger": "truck",
    "Lkw mit Anhänger": "truck",
    "Bus": "bus",
    "Fahrrad ohne Anhänger": "bicycle",
    "Fahrrad mit Anhänger": "bicycle",
    "Lastenrad": "bicycle"
}

In [379]:
#nicht mehrfach ausführen!
df_gt["Class"] = df_gt["Class"].map(eq_tb)

In [380]:
#Sportieren nach Zeit
df_eval = sort_time(df_eval)
df_gt = sort_time(df_gt)

In [381]:
t_min = pd.to_datetime("2022-10-17 13:20:00")
t_max = pd.to_datetime("2022-10-17 13:25:00")

df_11 = get_dfrange(df_eval, t_min, t_max)
df_12 = get_dfrange(df_gt, t_min, t_max)

In [401]:
t_min = df_gt["DateTime"].min()
t_max = df_gt["DateTime"].max() + pd.Timedelta(seconds=1)
t_int = pd.Timedelta(minutes=5, seconds=0) #evaluierungsinterval !

t_end = t_min
while t_end <=(t_max - t_int):
    t_start = t_end
    t_end = t_start + t_int
    print(t_end)
    

2022-10-17 13:20:00
2022-10-17 13:25:00
2022-10-17 13:30:00


In [432]:
t_start = pd.to_datetime("2022-10-17 13:26:35")
t_end = pd.to_datetime("2022-10-17 13:26:36")
#print(t_end)
df1 = get_df_range(df_gt,t_start,t_end)
get_value_counts(df1, gates, 3)
#df_gt[(df_gt["DateTime"] >= t_start) & (df_gt["DateTime"] < t_end)]

Class 
car       1
person    1
dtype: int64

In [426]:
gates[3]

'FG1'